In [2]:
import pandas as pd
import plotly.express as px
import polars as pl

In [14]:
df=pl.read_parquet("./data/parquet/users_500000.parquet")
df

user_id,book_id
i64,i64
0,948
0,947
0,946
0,945
0,944
…,…
1033,111847
1033,181478
1033,181477


# Lines

In [16]:
df = pd.read_csv("./benchmark_results_20250216.csv", sep=",", header=None, names=["engine", "file", "size","run", "time"])
df

,engine,file,size,run,time
0,pandas_cudf,csv,500000,1,0.153022
1,pandas_cudf,csv,500000,2,0.005645
2,pandas_cudf,csv,500000,3,0.006399
3,pandas_cudf,csv,500000,4,0.005191
4,pandas_cudf,csv,500000,5,0.005468
...,...,...,...,...,...
3127,spark,csv,200000000,8,0.055526
3128,spark,csv,200000000,9,0.047953
3129,spark,csv,200000000,10,0.046837
3130,spark,csv,200000000,11,0.055455


In [17]:
df_max = df.groupby(['engine','file','size']).agg({'time':'max'})
df_max['min_max'] = 1
df_min = df.groupby(['engine','file','size']).agg({'time':'min'})
df_min['min_max'] = 1
df_min_max = pd.concat([df_min,df_max])
df_min_max = df_min_max.reset_index()
df_min_max

,engine,file,size,time,min_max
0,pandas,csv,500000,0.041279,1
1,pandas,csv,1000000,0.071730,1
2,pandas,csv,5000000,0.420342,1
3,pandas,csv,10000000,0.853514,1
4,pandas,csv,15000000,1.344369,1
...,...,...,...,...,...
515,spark,parquet,100000000,0.976232,1
516,spark,parquet,125000000,0.976109,1
517,spark,parquet,150000000,0.979737,1
518,spark,parquet,175000000,1.012710,1


In [18]:
df_filtered = (
    df
    .merge(df_min_max, on=['engine','file','size','time'], how='left')
)

df_filtered = df_filtered[df_filtered['min_max'].isnull()]
df_filtered = df_filtered.drop('min_max', axis=1)
df_filtered

,engine,file,size,run,time
1,pandas_cudf,csv,500000,2,0.005645
2,pandas_cudf,csv,500000,3,0.006399
3,pandas_cudf,csv,500000,4,0.005191
4,pandas_cudf,csv,500000,5,0.005468
5,pandas_cudf,csv,500000,6,0.005985
...,...,...,...,...,...
3126,spark,csv,200000000,7,0.061613
3127,spark,csv,200000000,8,0.055526
3128,spark,csv,200000000,9,0.047953
3130,spark,csv,200000000,11,0.055455


In [19]:
df_mean = (
    df_filtered
    .groupby(['engine','file','size'])
    .agg(
        {
            'time':'mean'
        }
    )
)
df_mean = df_mean.reset_index()
df_mean['engine_file'] = df_mean['engine']+'-'+df_mean['file']
df_mean

,engine,file,size,time,engine_file
0,pandas,csv,500000,0.044428,pandas-csv
1,pandas,csv,1000000,0.076757,pandas-csv
2,pandas,csv,5000000,0.424388,pandas-csv
3,pandas,csv,10000000,0.855994,pandas-csv
4,pandas,csv,15000000,1.358373,pandas-csv
...,...,...,...,...,...
255,spark,parquet,100000000,0.035743,spark-parquet
256,spark,parquet,125000000,0.035751,spark-parquet
257,spark,parquet,150000000,0.035297,spark-parquet
258,spark,parquet,175000000,0.036641,spark-parquet


In [20]:
fig = px.line(
    df_mean,
    x='size',
    y='time',
    color='engine_file',
    height=800,
    width=1000,
    title='Average execution time - 10 runs',
    markers=True,
     labels={
        'time': 'Time (s)',
        'size': '# of lines',
        'engine_file': 'Engine - File'
    }
)

fig.show()

# Goodreads Db

In [21]:
df = pd.read_csv("./benchmark_results_goodreads_20250216.csv", sep=",", header=None, names=["engine", "file", "run", "time"])
df

,engine,file,run,time
0,pandas_cudf,csv,1,34.958037
1,pandas_cudf,csv,1,7.129519
2,pandas_cudf,csv,2,11.408551
3,pandas_cudf,csv,3,7.891881
4,pandas_cudf,csv,4,7.713310
...,...,...,...,...
116,spark,parquet,8,0.033622
117,spark,parquet,9,0.031467
118,spark,parquet,10,0.030925
119,spark,parquet,11,0.034351


In [22]:
df_max = df.groupby(['engine','file']).agg({'time':'max'})
df_max['min_max'] = 1
df_min = df.groupby(['engine','file']).agg({'time':'min'})
df_min['min_max'] = 1
df_min_max = pd.concat([df_min,df_max])
df_min_max = df_min_max.reset_index()
df_min_max

,engine,file,time,min_max
0,pandas,csv,26.473442,1
1,pandas,parquet,5.107713,1
2,pandas_cudf,csv,7.129519,1
3,pandas_cudf,parquet,2.532946,1
4,polars,csv,3.780570,1
5,polars,parquet,3.091340,1
6,polars_cudf,csv,1.022787,1
7,polars_cudf,parquet,0.306133,1
8,spark,csv,0.042252,1
9,spark,parquet,0.030925,1


In [23]:
df_filtered = (
    df
    .merge(df_min_max, on=['engine','file','time'], how='left')
)

df_filtered = df_filtered[df_filtered['min_max'].isnull()]
df_filtered = df_filtered.drop('min_max', axis=1)
df_filtered

,engine,file,run,time
2,pandas_cudf,csv,2,11.408551
3,pandas_cudf,csv,3,7.891881
4,pandas_cudf,csv,4,7.713310
5,pandas_cudf,csv,5,7.759629
6,pandas_cudf,csv,6,7.772991
...,...,...,...,...
115,spark,parquet,7,0.035691
116,spark,parquet,8,0.033622
117,spark,parquet,9,0.031467
119,spark,parquet,11,0.034351


In [24]:
df_mean = (
    df_filtered
    .groupby(['engine','file'])
    .agg(
        {
            'time':'mean'
        }
    )
)
df_mean = df_mean.reset_index()
df_mean['engine_file'] = df_mean['engine']+'-'+df_mean['file']
df_mean

,engine,file,time,engine_file
0,pandas,csv,28.676856,pandas-csv
1,pandas,parquet,5.234514,pandas-parquet
2,pandas_cudf,csv,8.046553,pandas_cudf-csv
3,pandas_cudf,parquet,3.135356,pandas_cudf-parquet
4,polars,csv,3.801188,polars-csv
5,polars,parquet,3.133524,polars-parquet
6,polars_cudf,csv,1.074315,polars_cudf-csv
7,polars_cudf,parquet,0.340801,polars_cudf-parquet
8,spark,csv,0.061850,spark-csv
9,spark,parquet,0.038362,spark-parquet


In [25]:
fig = px.bar(
    df_mean,
    x='file',
    y='time',
    color='engine',
    height=800,
    width=1000,
    title='Average execution time - 10 runs',
    labels={
        'time': 'Time (s)',
        'file': 'File',
        'engine': 'Engine'
    },
    barmode="group",
    text_auto='.2f'
)

fig.show()

In [33]:
df_metrics = (
    df_filtered
    .groupby(['engine','file'])
    .agg(
        mean = ('time','mean'),
        stddev = ('time', 'std'),
        median = ('time','median')
        )
)
df_metrics = df_metrics.reset_index()
df_metrics

,engine,file,mean,stddev,median
0,pandas,csv,28.676856,0.241594,28.730991
1,pandas,parquet,5.234514,0.246346,5.141955
2,pandas_cudf,csv,8.046553,1.118021,7.713310
3,pandas_cudf,parquet,3.135356,0.248288,3.052279
4,polars,csv,3.801188,0.023988,3.793273
5,polars,parquet,3.133524,0.027382,3.120671
6,polars_cudf,csv,1.074315,0.140156,1.030304
7,polars_cudf,parquet,0.340801,0.100387,0.308305
8,spark,csv,0.061850,0.011694,0.065556
9,spark,parquet,0.038362,0.007237,0.035753


In [38]:
df_metrics['Rate'] = round(df_metrics["mean"]/0.038362,2)
df_metrics.sort_values('Rate')

,engine,file,mean,stddev,median,Rate
9,spark,parquet,0.038362,0.007237,0.035753,1.00
8,spark,csv,0.061850,0.011694,0.065556,1.61
7,polars_cudf,parquet,0.340801,0.100387,0.308305,8.88
6,polars_cudf,csv,1.074315,0.140156,1.030304,28.00
5,polars,parquet,3.133524,0.027382,3.120671,81.68
3,pandas_cudf,parquet,3.135356,0.248288,3.052279,81.73
4,polars,csv,3.801188,0.023988,3.793273,99.09
1,pandas,parquet,5.234514,0.246346,5.141955,136.45
2,pandas_cudf,csv,8.046553,1.118021,7.713310,209.75
0,pandas,csv,28.676856,0.241594,28.730991,747.53
